# Notebook 6a: Advection test 



We'll look at an advection-diffusion problem to introduce the idea of time-dependence. 



<!-- $\eta$ is viscosity, $p$ is pressure, $\rho_0$ is a reference density, $\alpha$ is thermal expansivity, and $T$ is the temperature. Here we explicitly express density variations in terms of temperature variations.

Thermal evolution is given by
$$
\frac{\partial T}{\partial t} - \mathbf{u}\cdot\nabla T = \kappa \nabla^2 T 
$$
where the velocity, $\mathbf{u}$ is the result of the Stokes flow calculation. $\kappa$ is the thermal diffusivity (compare this with Notebook 4).  

The starting point is our previous notebook where we solved for Stokes
flow in a cylindrical annulus geometry. We then add an advection-diffusion 
solver to evolve temperature. The Stokes buoyancy force is proportional to the
temperature anomaly, and the velocity solution is fed back into the 
temperature advection term. The timestepping loop is written by
hand because usually you will want to do some analysis or output some checkpoints.

To read more about the applications of simple mantle convection models like this one, see (for example) Schubert et al, 2001. -->



In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import numpy as np
import sympy
import underworld3 as uw

In [3]:
res = 16

mesh = uw.meshing.UnstructuredSimplexBox(
    cellSize=1 / res, regular=False, qdegree=3, refinement=0
)

# Coordinate directions etc
x, y = mesh.CoordinateSystem.X

nsteps = 10
kappa = 1.0  # diffusive constant

velocity = 1.0e-3


t_start = 1e-4
t_end = 0.01

u, t, x, x0, x1 = sympy.symbols("u, t, x, x0, x1")


U_a_x = 0.5 * (
    sympy.erf((x1 - x + (u * t)) / (2 * sympy.sqrt(kappa * t)))
    + sympy.erf((-x0 + x - (u * t)) / (2 * sympy.sqrt(kappa * t)))
)


In [4]:
# Create mesh vars
v = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=1)
T = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)

# #### Create the advDiff solver

adv_diff = uw.systems.AdvDiffusion(
    mesh,
    u_Field=T,
    V_fn=v.sym,
)

# ### Set up properties of the adv_diff solver
# - Constitutive model (Diffusivity)
# - Boundary conditions
# - Internal velocity
# - Initial temperature distribution

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = kappa

### fix temp of top and bottom walls
adv_diff.add_dirichlet_bc(0.0, "Left")
adv_diff.add_dirichlet_bc(0.0, "Right")

# with mesh.access(v):
#     # initialise fields
#     # v.data[:,0] = -1*v.coords[:,1]
#     v.data[:, 1] = velocity

v.array[:,:,:] = np.array(((velocity, 0),))

In [5]:
adv_diff.estimate_dt()

array(0.00055668)

In [6]:
U_start = U_a_x.subs({u: velocity, t: t_start, x: mesh.X[0], x0: 0.4, x1: 0.6})

T.array = uw.function.evaluate(U_start, T.coords)

model_time = t_start

#### Solve
dt = 5*float(adv_diff.estimate_dt())

while model_time < t_end:
    print(model_time)
    
    if model_time + dt > t_end:
        dt = t_end - model_time

    ### diffuse through underworld
    adv_diff.solve(timestep=dt)

    model_time += dt

sample_x = np.arange(0, 1, mesh.get_min_radius())
sample_y = np.zeros_like(sample_x) + 0.5
sample_points = np.column_stack([sample_x, sample_y])

### compare UW and 1D numerical solution
T_UW = uw.function.evaluate(T.sym[0], sample_points).squeeze()

U_end = U_a_x.subs({u: velocity, t: model_time, x: mesh.X[0], x0: 0.4, x1: 0.6})
T_analytical = uw.function.evaluate(U_end, sample_points).squeeze()

### moderate atol due to evaluating onto points
# assert np.allclose(T_UW, T_analytical, atol=0.05)

0.0001
0.002883424250566981
0.005666848501133963
0.008450272751700945


In [7]:
T_UW-T_analytical

array([-0.00232731, -0.00377359, -0.00603793, -0.00945676, -0.014431  ,
       -0.02141938, -0.03090818, -0.04329841, -0.05880075, -0.07715727,
       -0.09742942, -0.11755613, -0.13383545, -0.14061096, -0.12996988,
       -0.09298964, -0.02235222,  0.07456375,  0.17344799,  0.24900089,
        0.29437222,  0.31208441,  0.30422762,  0.27002766,  0.20576951,
        0.11411981,  0.01257002, -0.06988467, -0.11933591, -0.13907051,
       -0.13793853, -0.12452336, -0.10537893, -0.08483988, -0.06557483,
       -0.0489014 , -0.03530892, -0.02473935, -0.01684397, -0.01115079,
       -0.00718809, -0.00452804, -0.00280285])

In [10]:
# visualise it


if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, T.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(T)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, T.sym)
    pvmesh_t.point_data["Ta"] = vis.scalar_fn_to_pv_points(pvmesh_t, U_end)
    pvmesh_t.point_data["dT"] = pvmesh_t.point_data["Ta"] - pvmesh_t.point_data["T"]


    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=True,
        # clim=(0,1),
    )


    pl.export_html("html5/box_advection_plot.html")
    # pl.show(cpos="xy", jupyter_backend="trame")

In [11]:
#| fig-cap: "Interactive Image: Convection model output"
from IPython.display import IFrame

IFrame(src="html5/box_advection_plot.html", width=500, height=400)